# Set Origin and Destination

Set the origin and the destination of usual route.

In [17]:
origin = (13.7828824,100.4706708)
destin = (13.7338328,100.5274467)

# Get FourSquare Venues from Origin and Destination

FourSquare API Key

In [18]:
# secret key
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version

GOOGLE_KEY = ''

In [19]:
# function for calling FourSquare API

import requests

def call4sq(coord):
	VENUE_TYPE = 'explore'  # explore trending
	section = 'arts'  # sights arts outdoors
	lat = coord[0]
	lng = coord[1]
	radius = 10000
	LIMIT = 100

	url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&section={}&ll={},{}&radius={}&limit={}'.format(
		VENUE_TYPE,
		CLIENT_ID, 
		CLIENT_SECRET, 
		VERSION,
		section,
		lat, 
		lng, 
		radius, 
		LIMIT)
	return requests.get(url).json()  

In [20]:
# function for converting JSON to DataFrame

import pandas as pd
from pandas.io.json import json_normalize

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def json2df(json):
	venues = json['response']['groups'][0]['items']
	nv = json_normalize(venues) # flatten JSON
	# filter columns
	filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.id',]
	nv = nv.loc[:, filtered_columns]

	# filter the category for each row
	nv['venue.categories'] = nv.apply(get_category_type, axis=1)

	# clean columns
	nv.columns = [col.split(".")[-1] for col in nv.columns]

	nv = nv[nv.categories != 'Multiplex']
	nv = nv[(nv.categories.str.contains('Theater')) == False]
	return nv

Find venues near the origin and the destination.  Combine the result into one DataFrame.

In [21]:
df1 = json2df(call4sq(origin)) # LPN P
df2 = json2df(call4sq(destin)) # CU
df1.set_index('id', inplace=True)
df2.set_index('id', inplace=True)
print(df1.shape)
print(df2.shape)

(74, 4)
(67, 4)


In [22]:
df = pd.concat([df1, df2]).drop_duplicates()
print(df.shape)
df.head()

(89, 4)


,name,categories,lat,lng
id,,,,
4dfb6e666284607895334ba5,Adhere the 13th Blues Bar,Jazz Club,13.763043,100.498839
52b6b6f1498e07724edcc2b4,Pipit Banglamphu (พิพิธบางลำพู),History Museum,13.763533,100.496432
4d20393a0901721e6ffc90a5,พระที่นั่งพุทไธสวรรย์,History Museum,13.757895,100.492100
50c015c5e4b093aa4baf2ef9,Siriraj Phimukhsthan Museum (พิพิธภัณฑ์ศิริราช...,History Museum,13.759675,100.487052
4c8882ac94f5a093117fd01e,Princess Maha Chakri Sirindhorn Anthropology C...,History Museum,13.783951,100.458725


# Calculate Detour Distance



In [23]:
!pip install -U googlemaps

Requirement already up-to-date: googlemaps in ./common/.virtualenv/python3/lib/python3.5/site-packages (3.0.2)


In [24]:
import googlemaps

gmaps = googlemaps.Client(key=GOOGLE_KEY)

In [25]:
# find distance and duration
def dist_dura(a, b):
    result = gmaps.distance_matrix(a, b, 'driving')
    return (int(result['rows'][0]['elements'][0]['distance']['value']), 
            int(result['rows'][0]['elements'][0]['duration']['value']))

# find detour distance and duration
def detour_dist_dura(venue, origin, destin):
    origin = '{},{}'.format(origin[0], origin[1])
    destin = '{},{}'.format(destin[0], destin[1])
    r0 = dist_dura(origin, venue) 
    r1 = dist_dura(venue, destin)
    return (r0[0] + r1[0], r0[1] + r1[1])

Find detour distance and duration of all rows.

In [26]:
s = df.lat.astype(str) + ',' + df.lng.astype(str)
s = s.apply(detour_dist_dura, args=(origin, destin))
s.head()

id
4dfb6e666284607895334ba5    (12609, 1856)
52b6b6f1498e07724edcc2b4    (12382, 1804)
4d20393a0901721e6ffc90a5    (13827, 2195)
50c015c5e4b093aa4baf2ef9    (16660, 2100)
4c8882ac94f5a093117fd01e    (16863, 1650)
dtype: object

Append the distances and durations to the DataFrame

In [27]:
df['distance'] = s.apply(lambda x : x[0])
df['duration'] = s.apply(lambda x : x[1])
df.sort_values('duration', inplace=True)
df.head()

,name,categories,lat,lng,distance,duration
id,,,,,,
4dd0d8c1c65bdac7139b737e,Grasshopper Adventures,Tour Provider,13.757171,100.500193,12682,1507
4b0587faf964a5204aaa22e3,หอศิลป์ สมเด็จพระนางเจ้าสิริกิติ์ พระบรมราชินี...,Art Gallery,13.756267,100.504884,12682,1508
4d61b57a29ef236acb38b459,Dusit Zoo Gallery,Art Gallery,13.774097,100.517401,14026,1601
4fcb2c67e4b023688a07e1d4,Reptile Exhibit (ห้องแสดงสัตว์เลื้อยคลาน),Zoo Exhibit,13.772691,100.518013,14058,1611
4f10ce5ce4b0042053b4bb63,พิพิธภัณฑ์สำนักงานตำรวจแห่งชาติ,Museum,13.768063,100.510269,13636,1634


In [28]:
distance, duration = dist_dura('{},{}'.format(origin[0], origin[1]), '{},{}'.format(destin[0], destin[1]))
df['dt_min'] = (df['duration'] - duration) / 60

# Visualization

In [29]:
#!conda install -c conda-forge folium=0.5.0 --yes 
!pip install folium

In [30]:
import folium # map rendering library
from ipywidgets import interact

In [31]:
# filter the DataFrame df by detour duration
def detour(minutes):  # detour(5)  0 to 5 minutes detour
    f1 = (df.duration - duration) < minutes * 60
    f2 = (minutes - 5) * 60 < (df.duration - duration)
    return df[f1 & f2]

In [32]:
# draw interactive map
def add_marker(mp, lat, lng, label):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(mp)


def draw_map(minutes):
    center_lat = (origin[0] + destin[0]) / 2
    center_lng = (origin[1] + destin[1]) / 2
    mp = folium.Map(location=[center_lat, center_lng], zoom_start=13)

    add_marker(mp, origin[0], origin[1], "Origin")
    add_marker(mp, destin[0], destin[1], "Destination")
    
    dtdf = detour(minutes)
    
    print(dtdf.shape)
    dtdf.apply(lambda row : folium.CircleMarker(
        [row['lat'], row['lng']],
        radius=5,
        popup='{} [{:.2f} mins]'.format(row['name'][0:32].replace("'", ' '), row['dt_min']),
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(mp), axis=1)
                          
    return mp
         
interact(draw_map, minutes=(5, 60, 5))

(4, 7)


<function __main__.draw_map>